In [1]:
import nibabel as nib
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
import tifffile as tiff
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import random

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Concatenate

from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras import backend as K
from keras.losses import binary_crossentropy
import tensorflow as tf


In [4]:
image_directory = 'MRI/Anatomical_mag_echo5/'
mask_directory = 'MRI/whole_liver_segmentation/'

image_dataset = []  
mask_dataset = []
sliced_image_dataset = []
sliced_mask_dataset = []
image_filenames =[]
image_slices_filenames =[]
mask_filenames =[]
mask_slices_filenames =[]


images = os.listdir(image_directory)
images.sort()
for i, image_name in enumerate(images):    
    if (image_name.split('.')[1] == 'nii'):
        image = nib.load(image_directory+image_name)
        image = np.array(image.get_fdata())
        image_dataset.append(np.array(image))


        image_filenames.append(os.path.splitext(image_name)[0]) 

masks = os.listdir(mask_directory)
masks.sort()
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'nii'):
        image = nib.load(mask_directory+image_name)
        image = np.array(image.get_fdata())
        mask_dataset.append(np.array(image))

        mask_filenames.append(os.path.splitext(image_name)[0]) 


for i in range(len(image_dataset)):
    for j in range(image_dataset[i].shape[2]):
        sliced_image_dataset.append(image_dataset[i][:,:,j])
        
        image_slice_id = f'{image_filenames[i]}-slice{j}'
        image_slices_filenames.append(image_slice_id)


for i in range(len(mask_dataset)):
    for j in range(mask_dataset[i].shape[2]):
        if mask_filenames[i] == 'f_3325' and j==31:
            continue
        else:
            sliced_mask_dataset.append(mask_dataset[i][:,:,j])
            mask_slice_id = f'{mask_filenames[i]}-slice{j}'
            mask_slices_filenames.append(mask_slice_id)




In [5]:
batch_size = 16 # The batch size to use when training the model
image_size = (224, 224)  # The size of the images

import tensorflow as tf
import tifffile as tiff
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import random

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Concatenate,Dropout

from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Building the SegNet Model



# Building the SegNet Model

def encoder_block(input_tensor, n_filters, kernel_size=3, batchnorm=True, dropout_rate=0.1):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if dropout_rate>0:
        x = Dropout(dropout_rate)(x)



    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def decoder_block(input_tensor, skip_tensor, n_filters, kernel_size=3, batchnorm=True, dropout_rate=0.1):
    x = UpSampling2D(size=(2, 2), interpolation='bilinear')(input_tensor)
    x = Concatenate()([x, skip_tensor])
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)



    if dropout_rate > 0:
        x = Dropout(dropout_rate)(x)



    
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def get_segnet_model(input_img, n_filters=64, n_classes=1, dropout=0.1, batchnorm=True):
    # Contracting Path (encoder)
    c1 = encoder_block(input_img, n_filters * 1, kernel_size=3, batchnorm=batchnorm, dropout_rate=dropout)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = encoder_block(p1, n_filters * 2, kernel_size=3, batchnorm=batchnorm, dropout_rate=dropout)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = encoder_block(p2, n_filters * 4, kernel_size=3, batchnorm=batchnorm, dropout_rate=dropout)
    p3 = MaxPooling2D((2, 2))(c3)
    
    c4 = encoder_block(p3, n_filters * 8, kernel_size=3, batchnorm=batchnorm, dropout_rate=dropout)
    p4 = MaxPooling2D((2, 2))(c4)
    
    # Expanding Path (decoder)
    u6 = decoder_block(c4, c3, n_filters * 4, kernel_size=3, batchnorm=batchnorm, dropout_rate=dropout)
    u7 = decoder_block(u6, c2, n_filters * 2, kernel_size=3, batchnorm=batchnorm, dropout_rate=dropout)
    u8 = decoder_block(u7, c1, n_filters * 1, kernel_size=3, batchnorm=batchnorm, dropout_rate=dropout)
    
    # Output layer
    output_img = Conv2D(n_classes, (1, 1), activation='sigmoid')(u8)
    
    return Model(inputs=input_img, outputs=output_img)


In [6]:
input_img = Input((224, 224, 1), name='img')
model = get_segnet_model(input_img)

model.load_weights(f'best_model114.keras')


In [8]:
 #5 test images 
for i in range(3151):
    test_img_number = i
    test_img = sliced_image_dataset[test_img_number]
    ground_truth = sliced_mask_dataset[test_img_number]
        #test_img_norm = test_img[:,:,0][:,:,None]
    test_img_input = np.expand_dims(test_img, axis=0)
    prediction = (model.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8)

    original_image_normalized = ground_truth.astype(float) / np.max(ground_truth)
    colored_mask = plt.get_cmap('jet')(prediction / np.max(prediction))
    alpha = 0.5 
    colored_mask[..., 3] = np.where(prediction > 0, alpha, 0)
    id = mask_slices_filenames[i]

      
        
    
    plt.figure(figsize=(16, 8))
    plt.subplot(141)
    plt.title('Testing Image')
    plt.imshow(test_img[:,:], cmap='gray')
    plt.subplot(142)
    plt.title('Testing Mask')
    plt.imshow(ground_truth[:,:], cmap='gray')
    plt.subplot(143)
    plt.title('Prediction on test image')
    plt.imshow(prediction, cmap='gray')
    plt.subplot(144)
    plt.title("Overlayed Images")
    plt.imshow(original_image_normalized, cmap='gray')
    plt.imshow(colored_mask, cmap='jet')
    plt.savefig(f'test_pred/{id}.png')
    plt.close()

1/1 [==============================] - 0s 358ms/step


C:\Users\Mittal\AppData\Local\Temp\ipykernel_18176\1175096469.py:10: RuntimeWarning: invalid value encountered in divide
  original_image_normalized = ground_truth.astype(float) / np.max(ground_truth)
C:\Users\Mittal\AppData\Local\Temp\ipykernel_18176\1175096469.py:11: RuntimeWarning: invalid value encountered in divide
  colored_mask = plt.get_cmap('jet')(prediction / np.max(prediction))


1/1 [==============================] - 0s 354ms/step
